# BAR CHART IN D3

## INTRODUCTION 

This is code continuation that is based on the blog : 
    
    
The notebook is a simple example of us D3 and how to make updates.

The link to bl.ock is here : https://bl.ocks.org/mbostock/3885304

Please refer to the blog for the explanation of this note book .

In [2]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [3]:
# Get the D3 host locally. 
HTML('<script src="./d3.min.js"></script>')

In [29]:
#HTML templet
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [5]:
# Css templet
css_text = '''

'''

In [30]:
#Java script templet
js_text1 = '''

d3 = require("d3@5")

width = 932
radius = width / 2


data = require("@observablehq/flare")


tree = data => d3.tree()
    .size([2 * Math.PI, radius])
    .separation((a, b) => (a.parent == b.parent ? 1 : 2) / a.depth)
  (d3.hierarchy(data))


chart = {
  const root = tree(data);
  
  const svg = d3.select("#graph-div").append("svg")
      .style("width", "100%")
      .style("height", "auto")
      .style("padding", "10px")
      .style("box-sizing", "border-box")
      .style("font", "10px sans-serif");
  
  const g = svg.append("g");
    
  const link = g.append("g")
      .attr("fill", "none")
      .attr("stroke", "#555")
      .attr("stroke-opacity", 0.4)
      .attr("stroke-width", 1.5)
    .selectAll("path")
    .data(root.links())
    .enter().append("path")
      .attr("d", d3.linkRadial()
          .angle(d => d.x)
          .radius(d => d.y));
  
  const node = g.append("g")
      .attr("stroke-linejoin", "round")
      .attr("stroke-width", 3)
    .selectAll("g")
    .data(root.descendants().reverse())
    .enter().append("g")
      .attr("transform", d => `
        rotate(${d.x * 180 / Math.PI - 90})
        translate(${d.y},0)
      `);
  
  node.append("circle")
      .attr("fill", d => d.children ? "#555" : "#999")
      .attr("r", 2.5);
  
  node.append("text")
      .attr("dy", "0.31em")
      .attr("x", d => d.x < Math.PI === !d.children ? 6 : -6)
      .attr("text-anchor", d => d.x < Math.PI === !d.children ? "start" : "end")
      .attr("transform", d => d.x >= Math.PI ? "rotate(180)" : null)
      .text(d => d.data.name)
    .clone(true).lower()
      .attr("stroke", "white");
  
  document.body.appendChild(svg.node());

  const box = g.node().getBBox();

  svg.remove()
      .attr("width", box.width)
      .attr("height", box.height)
      .attr("viewBox", `${box.x} ${box.y} ${box.width} ${box.height}`);

  return svg.node();
}

'''

temp1 = Template(js_text1)

In [21]:
js_text_template_2 = Template('''

var bars = svg.selectAll(".bar").data($data);
  
bars
  .transition()
    .attr("y", function(d) { return y(d.y); })
    .attr("height", function(d) { return height - y(d.y); });

''')

In [22]:
data = pd.DataFrame({'letter': ['A','B','C','D'], 'y': [1,1,1,1]})
data.head()


data_new = {
    "name": "test",
    "children": [
        { "name": "test2",
         "children": []},
        {"name" : "test3",
        "children":[]}
    ]
}

In [23]:
json.dumps(data_new)

'{"name": "test", "children": [{"name": "test2", "children": []}, {"name": "test3", "children": []}]}'

In [31]:
# js_text = js_text_template.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text1}))

In [18]:
data['y'] = [random.uniform(0,1) for d in data['y']]
js_text = js_text_template_2.substitute({'data': json.dumps(data.to_dict(orient='records'))})
HTML('<script>' + js_text + '</script>')

In [32]:
HTML("""<!DOCTYPE html>
<meta charset="utf-8">
<style>
.node circle {
  fill: #999;
}
.node text {
  font: 10px sans-serif;
}
.node--internal circle {
  fill: #555;
}
.node--internal text {
  text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
}
.link {
  fill: none;
  stroke: #555;
  stroke-opacity: 0.4;
  stroke-width: 1.5px;
}
</style>
<svg width="960" height="1060"></svg>
<script src="https://d3js.org/d3.v4.min.js"></script>
<script>
var svg = d3.select("svg"),
    width = +svg.attr("width"),
    height = +svg.attr("height"),
    g = svg.append("g").attr("transform", "translate(" + (width / 2 + 40) + "," + (height / 2 + 90) + ")");
var stratify = d3.stratify()
    .parentId(function(d) { return d.id.substring(0, d.id.lastIndexOf(".")); });
var tree = d3.tree()
    .size([2 * Math.PI, 500])
    .separation(function(a, b) { return (a.parent == b.parent ? 1 : 2) / a.depth; });
d3.csv("flare.csv", function(error, data) {
  if (error) throw error;
  var root = tree(stratify(data));
  var link = g.selectAll(".link")
    .data(root.links())
    .enter().append("path")
      .attr("class", "link")
      .attr("d", d3.linkRadial()
          .angle(function(d) { return d.x; })
          .radius(function(d) { return d.y; }));
  var node = g.selectAll(".node")
    .data(root.descendants())
    .enter().append("g")
      .attr("class", function(d) { return "node" + (d.children ? " node--internal" : " node--leaf"); })
      .attr("transform", function(d) { return "translate(" + radialPoint(d.x, d.y) + ")"; });
  node.append("circle")
      .attr("r", 2.5);
  node.append("text")
      .attr("dy", "0.31em")
      .attr("x", function(d) { return d.x < Math.PI === !d.children ? 6 : -6; })
      .attr("text-anchor", function(d) { return d.x < Math.PI === !d.children ? "start" : "end"; })
      .attr("transform", function(d) { return "rotate(" + (d.x < Math.PI ? d.x - Math.PI / 2 : d.x + Math.PI / 2) * 180 / Math.PI + ")"; })
      .text(function(d) { return d.id.substring(d.id.lastIndexOf(".") + 1); });
});
function radialPoint(x, y) {
  return [(y = +y) * Math.cos(x -= Math.PI / 2), y * Math.sin(x)];
}
</script>""")